# Generate prompts from raw stimuli and instructions for different measurements
Author: Hening Wang
Date: 21.08.2023

## Generate prompts for Rating task

In [41]:
# Import relevant modules
import pandas as pd
import numpy as np


In [42]:
# Read original prompts table from Hu et al. (2023) as basis for modification
df_prompts = pd.read_csv('../prompts/Maxims_prompts_seed0_examples0.csv')
print(df_prompts.content[0])
print(df_prompts.prompt[0])
df_prompts.head()

Bob is having a lunch meeting with his boss, Mr. James. Mr. James asks Bob: "Did you like the presentation that I gave at the board meeting yesterday?" Bob responds: "I cannot wait for our trip to Japan next week." Why has Bob responded like this?
Task: You will read short stories that describe everyday situations. Each story will be followed by a multiple-choice question. Read each story and choose the best answer. Your task is to decide why the character in the story responds in a certain way. The answer options are 1, 2, 3, or 4.

Scenario: Bob is having a lunch meeting with his boss, Mr. James. Mr. James asks Bob: "Did you like the presentation that I gave at the board meeting yesterday?" Bob responds: "I cannot wait for our trip to Japan next week." Why has Bob responded like this?
Options:
1) He is excited about the upcoming trip to Japan and does not want to talk about anything else.
2) He wants to avoid going to Japan next week and wants to discuss it with his boss. 
3) He disl

,phenomenon,item_id,content,randomized_option_order,randomized_labels_complex,randomized_violated_maxims,randomized_true_answer,original_labels_complex,original_violated_maxims,is_example,prompt,seed
0,MV,1,"Bob is having a lunch meeting with his boss, M...","[1, 4, 3, 2]","['IncorrectLiteral', 'IncorrectAssociate', 'Co...","['RelevanceMaxim', 'RelevanceMaxim', 'Relevanc...",3,"['IncorrectLiteral', 'IncorrectNonLiteral', 'C...","['RelevanceMaxim', 'RelevanceMaxim', 'Relevanc...",False,Task: You will read short stories that describ...,0
1,MV,2,"Richard came to visit his friend, Alex. Alex i...","[1, 4, 3, 2]","['IncorrectLiteral', 'IncorrectAssociate', 'In...","['RelevanceMaxim', 'RelevanceMaxim', 'Relevanc...",4,"['IncorrectLiteral', 'CorrectNonLiteral', 'Inc...","['RelevanceMaxim', 'RelevanceMaxim', 'Relevanc...",False,Task: You will read short stories that describ...,0
2,MV,3,Leslie and Jane are chatting at a coffee shop....,"[4, 1, 2, 3]","['IncorrectAssociate', 'IncorrectLiteral', 'Co...","['RelevanceMaxim', 'RelevanceMaxim', 'Relevanc...",3,"['IncorrectLiteral', 'CorrectNonLiteral', 'Inc...","['RelevanceMaxim', 'RelevanceMaxim', 'Relevanc...",False,Task: You will read short stories that describ...,0
3,MV,4,Kelsey is in a bad mood. Every minor thing irr...,"[4, 2, 1, 3]","['IncorrectAssociate', 'CorrectNonLiteral', 'I...","['RelevanceMaxim', 'RelevanceMaxim', 'Relevanc...",2,"['IncorrectLiteral', 'CorrectNonLiteral', 'Inc...","['RelevanceMaxim', 'RelevanceMaxim', 'Relevanc...",False,Task: You will read short stories that describ...,0
4,MV,5,"Mark is discussing Jane, a senior financial ma...","[2, 3, 1, 4]","['CorrectNonLiteral', 'IncorrectNonLiteral', '...","['RelevanceMaxim', 'RelevanceMaxim', 'Relevanc...",1,"['IncorrectLiteral', 'CorrectNonLiteral', 'Inc...","['RelevanceMaxim', 'RelevanceMaxim', 'Relevanc...",False,Task: You will read short stories that describ...,0


In [43]:
# Read the instructions for different prompts
file_path_instructions = 'raw_stimuli/MaximsInstructions_Rating.txt'

with open(file_path_instructions, 'r') as file:
    instructions = file.read()

print(instructions)

Task: You will read short stories that describe everyday situations. Each story will be followed by a description about the intention of one of the characters in that scenario giving a response. Read each story and associated descriptions. Your task is to decide how likely the character in the story responds in that way. The degree varies from very unlikely, unlikely, at chance, likely, very likely. Answer only a degree.


In [44]:
# Read options from the file

file_path_options = 'raw_stimuli/MaximsOptions.txt'

# Reading the content of the file into a DataFrame
column_names = ['item_number', 'option_number', 'option_text']
df_options = pd.read_csv(file_path_options, sep='\t', names=column_names)



In [45]:
# Get true answer with options and key
## Get keys and item_number pairs
keys_items_pairs = []
for i in range(len(df_prompts)):
    keys_items_pairs.append((df_prompts['item_id'][i], df_prompts['randomized_true_answer'][i]))


## Get true answer with key item pairs
true_answer = []
for item_number, option_number in keys_items_pairs:
    matching_option = df_options[
        (df_options['item_number'] == item_number) & 
        (df_options['option_number'] == option_number)
    ]['option_text']
    matching_option = "Description: " + matching_option
    true_answer.extend(matching_option.tolist())

In [50]:
# Combine instructions, scenarios and true anwsers together
new_prompts = []
for i in range(len(true_answer)):
    new_prompts.append(instructions + " " + "Scenario: " + df_prompts.content[i] + " " + true_answer[i])

# Generate new prompts
df_prompts.prompts = new_prompts

df_prompts.head()
df_prompts.to_csv("Maxims_prompts_Rating.csv", index=False)